# V2 Model Server (SKLearn)

In [1]:
import mlrun

> 2020-10-12 14:52:57,535 [warning] Failed resolving version info. Ignoring and using defaults


### Configuration and package dependencies

In [2]:
%nuclio config kind="serving"
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'serving'
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


In [3]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle v3io sklearn

### Serving class code

In [4]:
from cloudpickle import load
import numpy as np
from typing import List

In [5]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model(".pkl")
        self.model = load(open(model_file, "rb"))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body["inputs"])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [6]:
# mlrun: end-code

# Convert to function object

The sklearn-project generated one or more models that will be deployed in the server project `sklearn-servers`

In [7]:
models_path = "https://s3.wasabisys.com/iguazio/models/iris/model.pkl"

### Document and save (as a template)

In [16]:
fn = mlrun.code_to_function(
    "v2-model-server",
    description="generic sklearn model server",
    categories=["serving", "ml"],
    labels={"author": "yaronh", "framework": "sklearn"},
    code_output=".",
)
fn.spec.default_class = "ClassifierModel"
# print(fn.to_yaml())
fn.export()

> 2020-10-12 14:47:46,331 [info] function spec saved to path: function.yaml


### Configure and add model(s)

In [9]:
fn.add_model("mymodel", model_path=models_path)
# fn.verbose = True

# Test models locally (using a server emulator)

In [ ]:
# create an emulator (mock server) from the function configuration)
server = fn.to_mock_server(globals())

### Test against the iris dataset 

In [12]:
from sklearn.datasets import load_iris

iris = load_iris()
x = iris["data"].tolist()

In [13]:
result = server.test("/v2/models/mymodel/infer", {"inputs": x})
result.keys()

> 2020-10-12 14:44:19,297 [debug] router run model mymodel, op=infer
> 2020-10-12 14:44:19,297 [debug] router run model mymodel, op=infer


dict_keys(['id', 'model_name', 'outputs'])

##  Deploy server

In [14]:
fn.apply(mlrun.mount_v3io())
fn.deploy(project="v2-srv")

> 2020-10-12 14:44:20,662 [info] deploy started
[nuclio] 2020-10-12 14:44:21,772 (info) Build complete
[nuclio] 2020-10-12 14:44:29,852 (info) Function deploy complete
[nuclio] 2020-10-12 14:44:29,859 done updating v2-srv-v2-model-server, function address: 3.128.234.166:30830


'http://3.128.234.166:30830'

##  Test server

In [15]:
my_data = """{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}"""
fn.invoke("/v2/models/mymodel/infer", my_data)

{'id': 'bd688fa9-38cc-4a5d-95e8-1445bdd1520a',
 'model_name': 'mymodel',
 'outputs': [0, 2]}